In [ ]:
import json
with open('dataforchunk/lumberchunker.json', 'r', encoding='utf-8') as file:  
    qa_data = json.load(file)
len_sents=0
len_lists=0
for sentence in qa_data:
    len_sents+=len(sentence)
    len_lists+=1
len_sents/len_lists,len_sents

(176.38325239532747, 1343864)

In [ ]:
# Dynamically merge segmented files
import json  
import os
target_size=211
filename='multifieldqa_zh/qwen25_14B_set.json' 
chunk_path='dataforchunk/multifieldqa_zh/qwen25_14B_set.json'    

all_qa_data=[]
with open(chunk_path, 'r', encoding='utf-8') as cfile: 
    qa_data = json.load(cfile)

for item in qa_data: 
    all_qa_data=all_qa_data+item["gpt_output"]  # "final_chunks"  "gpt_output"
# all_qa_data=qa_data

len_sents=0
len_lists=0
for sentence in all_qa_data:
    len_sents+=len(sentence)
    len_lists+=1
print(len_sents/len_lists,len_sents)

merged_paragraphs = []  
current_paragraph = "" 
for i,paragraph in enumerate(all_qa_data):  
    if not isinstance(paragraph, str):
        if isinstance(paragraph, list):
            print(i,paragraph)
            paragraph = ' '.join(str(item) for item in paragraph)
    if len(current_paragraph) + len(paragraph) <= target_size:  
        # try:
        current_paragraph +=' '+paragraph  
        # except:
        #     print(i,paragraph)
    else:  
        merged_paragraphs.append(current_paragraph)  
        current_paragraph = paragraph  
if current_paragraph:  
    merged_paragraphs.append(current_paragraph)  

with open(filename, 'w', encoding='utf-8') as sfile:
    json.dump(merged_paragraphs, sfile, ensure_ascii=False, indent=4)

with open(filename, 'r', encoding='utf-8') as file:  
    qa_data = json.load(file)
len_sents=0
len_lists=0
for sentence in qa_data:
    len_sents+=len(sentence)
    len_lists+=1
len_sents/len_lists,len_sents

54.656591183537294 4390400


(178.53288892458437, 4445826)

In [ ]:
# Merge the chunk results of each document according to the specified length
def merge_short_texts(chunks, min_length=30, max_length=200):
    new_chunks = []
    for i,text in enumerate(chunks):
        if len(text) < min_length:
            if i==0:
                new_chunks.append(text)
            else:
                if len(new_chunks[-1]) + len(text) <= max_length:
                    new_chunks[-1] += text
                else:
                    new_chunks.append(text)
        else:
            new_chunks.append(text)
    return new_chunks

import json  
import os
root_dir = 'multifieldqa_zh/chunk_1.5all3_M_merge.json'
filename = 'dataforchunk/multifieldqa_zh/chunk_1.5all3_M.json'
 
min_length = 80
max_length = 225

with open(filename, 'r', encoding='utf-8') as file:  
    qa_data = json.load(file)
len_sents=0
len_lists=0
for item in qa_data:
    for sentence in item["final_chunks"]:
        len_sents+=len(sentence)
        len_lists+=1
print(len_sents/len_lists,len_sents)

all_data_chunk=[]
for item in qa_data: 
    all_data_chunk += merge_short_texts(item["final_chunks"], min_length, max_length)
    
with open(root_dir, 'w', encoding='utf-8') as sfile:
    json.dump(all_data_chunk, sfile, ensure_ascii=False, indent=4)


len_sents=0
len_lists=0
for sentence in all_data_chunk:
    len_sents+=len(sentence)
    len_lists+=1
print(len_sents/len_lists,len_sents)

126.18601969307194 1422495
178.61564540431945 1422495


In [ ]:
# Merge the chunk results of each document according to the specified length
def merge_short_texts(item, min_length=30, max_length=200):
    new_chunks = []
    new_outline = []
    new_core = []
    for i,text in enumerate(item["final_chunk"]):
        if len(text) < min_length:
            if i==0:
                new_chunks.append(text)
                new_outline.append(item["outline"][i])
                new_core.append(item["core_content"][i])
            else:
                if len(new_chunks[-1]) + len(text) <= max_length:
                    new_chunks[-1] += text
                else:
                    new_chunks.append(text)
                    new_outline.append(item["outline"][i])
                    new_core.append(item["core_content"][i])
        else:
            new_chunks.append(text)
            new_outline.append(item["outline"][i])
            new_core.append(item["core_content"][i])
    save = {"outline":new_outline,"core_content":new_core,"final_chunk":new_chunks}
    return save

import json  
import os
root_dir = 'multifieldqa_zh/ratio_7B.json'
filename = 'dataforchunk/multifieldqa_zh/ratio_7B.json'

min_length = 50
max_length = 250

with open(filename, 'r', encoding='utf-8') as file:  
    qa_data = json.load(file)
len_sents=0
len_lists=0
for item in qa_data:
    for sentence in item["final_chunk"]:
        len_sents+=len(sentence)
        len_lists+=1
print(len_sents/len_lists,len_sents)

all_data_chunk=[]
for item in qa_data: 
    all_data_chunk.append(merge_short_texts(item, min_length, max_length))
    
with open(root_dir, 'w', encoding='utf-8') as sfile:
    json.dump(all_data_chunk, sfile, ensure_ascii=False, indent=4)


len_sents=0
len_lists=0
for item in all_data_chunk:
    for sentence in item["final_chunk"]:
        len_sents+=len(sentence)
        len_lists+=1
print(len_sents/len_lists,len_sents)

## 几种不同分块方式

In [ ]:
root_dir = 'multifieldqa_zh/ratio_7B.json'
filename = 'dataforchunk/multifieldqa_zh/ratio_7B.json'

In [ ]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index.node_parser import SimpleNodeParser
import json
import os
import time

root_dir = 'raw/multifieldqa_zh'
all_data=[]
for root, dirs, files in os.walk(root_dir):  
    for file in files:
        documents = SimpleDirectoryReader(input_files=[os.path.join(root, file)]).load_data() 
        node_parser = SimpleNodeParser.from_defaults(
            chunk_size=281, chunk_overlap=0)
        nodes_tmp = node_parser.get_nodes_from_documents(documents, show_progress=True)

        nodes=[]
        for node in nodes_tmp:
            i=node.text
            nodes.append(i)
        all_data+=nodes  
with open('dataforchunk/multifieldqa_zh/llamaindex.json', 'w', encoding='utf-8') as sfile:
    json.dump(all_data, sfile, ensure_ascii=False, indent=4)
len_sents=0
len_lists=0
for sentence in all_data:
    len_sents+=len(sentence)
    len_lists+=1
print(len_sents/len_lists,len_sents)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

178.34745968722146 4401972


In [ ]:
import json
chunk_size=179
with open('raw/multifieldqa_zh/1.txt', 'r', encoding='utf-8') as file:  
    # 读取文件内容  
    content = file.read()  
# 打印文件内容  
chunks = [content[i:i+chunk_size] for i in range(0, len(content), chunk_size)] 
            
with open('dataforchunk/multifieldqa_zh/chunk_original.json', 'w', encoding='utf-8') as sfile:
    json.dump(chunks, sfile, ensure_ascii=False, indent=4)
len_sents=0
len_lists=0
for sentence in chunks:
    len_sents+=len(sentence)
    len_lists+=1
print(len_sents/len_lists,len_sents)

178.99882142566852 4404445


In [ ]:
from llama_index import SimpleDirectoryReader
from llama_index.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import json
import time

start_time = time.time() 
save_path='dataforchunk/multifieldqa_zh/semantic_75.json'
split_path='raw/multifieldqa_zh'
documents = SimpleDirectoryReader(split_path).load_data()

embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-base-zh-v1.5")  
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=75, embed_model=embed_model   
)                   # Chinese needs to modify the sentence segmentation function: split_by_my_zh

all_data=[]
for item in documents:
    nodes = splitter.get_nodes_from_documents([item], show_progress=True)

    for node in nodes:
        if node.text.strip() !='':
            all_data.append(node.text)
        
with open(save_path, 'w', encoding='utf-8') as sfile:
    json.dump(all_data, sfile, ensure_ascii=False, indent=4)
end_time = time.time()  

execution_time = end_time - start_time  
print(f"程序执行时间为: {execution_time} 秒")

len_sents=0
len_lists=0
for sentence in all_data:
    len_sents+=len(sentence)
    len_lists+=1
print(len_sents/len_lists,len_sents)

/users/u2023000898/envs/chunk/lib/python3.10/site-packages/llama_index/download/module.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.868 seconds.
Prefix dict has been built successfully.


Generating embeddings:   0%|          | 0/98288 [00:00<?, ?it/s]

程序执行时间为: 317.780415058136 秒
178.80218125584992 4393706


## 评估

In [ ]:
# -*- coding: utf-8 -*-
import evaluate
import os
import json
from tqdm import tqdm
import jieba
def bleu_score(
    continuation: str,
    reference: str,
    with_penalty = False
) -> float:
    f = lambda text: list(jieba.cut(text))
    bleu = evaluate.load('src/.cache/huggingface/bleu')
    results = bleu.compute(predictions=[continuation], references=[[reference]], tokenizer=f)
    
    bleu_avg = results['bleu']
    bleu1 = results['precisions'][0]
    bleu2 = results['precisions'][1]
    bleu3 = results['precisions'][2]
    bleu4 = results['precisions'][3]
    brevity_penalty = results['brevity_penalty']

    if with_penalty:
        return bleu_avg, bleu1, bleu2, bleu3, bleu4
    else:
        return 0.0 if brevity_penalty==0 else bleu_avg/brevity_penalty, bleu1, bleu2, bleu3, bleu4

def rougeL_score(
    continuation: str,
    reference: str
) -> float:
    f = lambda text: list(jieba.cut(text))
    rouge = evaluate.load('src/.cache/huggingface/rouge')
    results = rouge.compute(predictions=[continuation], references=[[reference]], tokenizer=f, rouge_types=['rougeL'])
    score = results['rougeL']
    return score

folder_path = 'dataforchunk/ccc_eval'
for filename in os.listdir(folder_path):
    # 检查文件是否以.json结尾
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        print(file_path)
        with open(file_path, 'r', encoding='utf-8') as file:  
            qa_data = json.load(file)
        score_list=[] 
        all_bleu_avg=0
        all_bleu1=0
        all_bleu2=0
        all_bleu3=0
        all_bleu4=0
        num=0
        for qa in tqdm(qa_data):
            query = qa['answer']
            llm_ans = qa['llm_ans']  

            rougeL=rougeL_score(llm_ans,query)
            bleu_avg, bleu1, bleu2, bleu3, bleu4=bleu_score(llm_ans,query,True)
            all_bleu_avg+=bleu_avg
            all_bleu1+=bleu1
            all_bleu2+=bleu2
            all_bleu3+=bleu3
            all_bleu4+=bleu4
            score_list.append(rougeL)
            num+=1
        print(filename,',avg_rougeL: ',sum(score_list)/len(score_list),flush=True)
        print(filename,',avg_bleu_avg: ',all_bleu_avg/num,flush=True)
        print(filename,',avg_bleu1: ',all_bleu1/num,flush=True)
        print(filename,',avg_bleu2: ',all_bleu2/num,flush=True)
        print(filename,',avg_bleu3: ',all_bleu3/num,flush=True)
        print(filename,',avg_bleu4: ',all_bleu4/num,flush=True)


In [ ]:
# -*- coding: utf-8 -*-
from nltk.translate.meteor_score import meteor_score
import jieba
import os
import json
from tqdm import tqdm

# 对文本进行分词并进行处理
def preprocess_text(text):
    return ' '.join(jieba.cut(text)).split()

folder_path = 'dataforchunk/ccc_eval'
for filename in os.listdir(folder_path):
    # 检查文件是否以.json结尾
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:  
            qa_data = json.load(file)
        score_list=[] 
        for qa in tqdm(qa_data):
            query = qa['answer']
            llm_ans = qa['llm_ans']  

            processed_reference = preprocess_text(query)
            processed_candidate = preprocess_text(llm_ans)

            score = meteor_score([processed_reference], processed_candidate)
            score_list.append(score)
        print(filename,',avg_sim: ',sum(score_list)/len(score_list),flush=True)

In [ ]:
import json
with open('dataforchunk/aaa/OmniEval_ratio_sc_7B_oec.json', 'r', encoding='utf-8') as file:  
    qa_data = json.load(file)
all_outline={}
all_core={}
for item in qa_data:
    all_outline['文档大纲：\n'+'\n'.join(item["outline"])]=item["final_chunk"]
    all_core['文档关键信息：\n'+'\n'.join(item["core_content"])]=item["final_chunk"]
with open('dataforchunk/aaa/a_OmniEval_ratio_sc_7B_oec.json', 'w', encoding='utf-8') as f:
    json.dump([all_outline,all_core], f, indent=4, ensure_ascii=False)

In [39]:
import json
with open('dataforchunk/ccc/OmniEval_ratio_sc_7B_oec.json', 'r', encoding='utf-8') as file:  
    qa_data = json.load(file)
all_core={}
for item in qa_data:
    for i,iit in enumerate(item["final_chunk"]):
        all_core['文本块关键信息：\n'+item["outline"][i].replace('\n',' ').strip()+'\n'+item["core_content"][i].replace('\n',' ').strip()]=item["final_chunk"]+item["core_content"]
with open('dataforchunk/ccc/a_OmniEval_ratio_sc_7B_oec.json', 'w', encoding='utf-8') as f:
    json.dump([all_core], f, indent=4, ensure_ascii=False)